In [1]:
import ROOT
import numpy as np
import pandas as pd
from my_library.common_analysis_tools import get_integrated_gluex1_data

Welcome to JupyROOT 6.24/04


see here: /work/halld/home/viducic/scripts/crosssection/kstar_rejection_dalitz_efficiency.ipynb

In [2]:
mass_low = 1.0 # GeV
mass_high = 2.5
step_size = 0.01


a0_mass_upper = 1.05


kaon_mass = 0.493677
kshort_mass = 0.497611
pion_mass = 0.139570

In [3]:
def check_if_a0_region(mass):
    return mass < a0_mass_upper


def get_weight_per_mass(mass_bin, draw_plots=False):
    phasespace_centermass = ROOT.TLorentzVector(0, 0, 0, mass_bin)
    phasespace_generator = ROOT.TGenPhaseSpace()
    phasespace_generator.SetDecay(phasespace_centermass, 3, np.array([kaon_mass, kshort_mass, pion_mass]))

    total = 0
    passed = 0

    full_dist = ROOT.TH1D(f"mass_dist_{mass_bin}", f"mass_dist_{mass_bin}", 100, 0.8, 2)
    passed_dist = ROOT.TH1D(f"passed_mass_dist_{mass_bin}", f"passed_mass_dist_{mass_bin}", 100, 0.8, 2)

    for _ in range(100000):
        weight = phasespace_generator.Generate()
        kaon = phasespace_generator.GetDecay(0)
        kshort = phasespace_generator.GetDecay(1)
        pion = phasespace_generator.GetDecay(2)

        a0_mass = (kaon + kshort).M()

        full_dist.Fill(a0_mass, weight)
        total += weight

        if check_if_a0_region(a0_mass):
            continue 

        passed_dist.Fill(a0_mass, weight)
        passed += weight

    if draw_plots:
        full_dist.SetLineColor(ROOT.kRed)
        passed_dist.SetLineColor(ROOT.kBlue)
        c = ROOT.TCanvas()
        c.Divide(2,1)
        c.cd(1)
        full_dist.Draw()
        c.cd(2)
        passed_dist.Draw()
        c.Update()
        c.SaveAs("/work/halld/home/viducic/plots/ps_a0/a0_{}.png".format(mass_bin))
        c.Close()

    return passed / total

In [4]:
eff_list = []
for mass_bin in np.arange(mass_low, mass_high+step_size, step_size):
    bin_center = round(mass_bin + step_size/2, 4)
    eff = get_weight_per_mass(bin_center, draw_plots=True)
    eff_list.append((bin_center, eff))

Info in <TCanvas::Print>: png file /work/halld/home/viducic/plots/ps_a0/a0_1.005.png has been created
Info in <TCanvas::Print>: png file /work/halld/home/viducic/plots/ps_a0/a0_1.015.png has been created
Info in <TCanvas::Print>: png file /work/halld/home/viducic/plots/ps_a0/a0_1.025.png has been created
Info in <TCanvas::Print>: png file /work/halld/home/viducic/plots/ps_a0/a0_1.035.png has been created
Info in <TCanvas::Print>: png file /work/halld/home/viducic/plots/ps_a0/a0_1.045.png has been created
Info in <TCanvas::Print>: png file /work/halld/home/viducic/plots/ps_a0/a0_1.055.png has been created
Info in <TCanvas::Print>: png file /work/halld/home/viducic/plots/ps_a0/a0_1.065.png has been created
Info in <TCanvas::Print>: png file /work/halld/home/viducic/plots/ps_a0/a0_1.075.png has been created
Info in <TCanvas::Print>: png file /work/halld/home/viducic/plots/ps_a0/a0_1.085.png has been created
Info in <TCanvas::Print>: png file /work/halld/home/viducic/plots/ps_a0/a0_1.095.p

In [5]:
df = pd.DataFrame(eff_list, columns=['mass', 'a0_cut_efficiency'])
df['a0_cut_efficiency'] = df['a0_cut_efficiency'].fillna(1)
df.to_csv(f"/work/halld/home/viducic/data/ps_a0_eff/a0_cut_efficiency_{step_size*1000}.csv", index=False)

In [6]:
def apply_efficiency_correction(hist, df):
    corrected_hist = hist.Clone()
    for i in range(1, corrected_hist.GetNbinsX()+1):
        mass_bin_center = round(hist.GetBinCenter(i),3)
        eff_df = df[df["mass_bin_center"] == round(mass_bin_center, 3)]["kstar_cut_efficiency"]
        if len(eff_df) == 0:
            continue
        efficiency = eff_df.values[0]
        corrected_hist.SetBinContent(i, hist.GetBinContent(i) / efficiency)
    
    corrected_hist.SetDirectory(0)

    return corrected_hist

In [7]:
# TODO: apply efficiency correction to the data and evaluate
hist = get_integrated_gluex1_data('pipkmks', 'all')

/work/halld/home/viducic/data/pipkmks/data/bestX2/pipkmks_flat_result_2018_spring.root
pipkmks_kstar_all_cut_beam_full_t_full;1
/work/halld/home/viducic/data/pipkmks/data/bestX2/pipkmks_flat_result_2018_fall.root
pipkmks_kstar_all_cut_beam_full_t_full;1
/work/halld/home/viducic/data/pipkmks/data/bestX2/pipkmks_flat_result_2017.root
pipkmks_kstar_all_cut_beam_full_t_full;1
